# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f963c761400>
├── 'a' --> tensor([[-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877]])
└── 'x' --> <FastTreeValue 0x7f963c761940>
    └── 'c' --> tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                        [-0.8656,  1.7485, -2.4316, -1.8778],
                        [-1.3291,  0.1364,  1.3137, -1.9218]])

In [4]:
t.a

tensor([[-0.0925, -0.6357,  1.1402],
        [-2.0423, -1.1758,  0.9877]])

In [5]:
%timeit t.a

79.1 ns ± 1.89 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f963c761400>
├── 'a' --> tensor([[-0.1101, -1.0954,  0.5485],
│                   [-0.1098,  0.3440,  0.8950]])
└── 'x' --> <FastTreeValue 0x7f963c761940>
    └── 'c' --> tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                        [-0.8656,  1.7485, -2.4316, -1.8778],
                        [-1.3291,  0.1364,  1.3137, -1.9218]])

In [7]:
%timeit t.a = new_value

89.5 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0925, -0.6357,  1.1402],
               [-2.0423, -1.1758,  0.9877]]),
    x: Batch(
           c: tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218]]),
       ),
)

In [10]:
b.a

tensor([[-0.0925, -0.6357,  1.1402],
        [-2.0423, -1.1758,  0.9877]])

In [11]:
%timeit b.a

75.3 ns ± 1.69 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3962,  0.1415, -0.6661],
               [ 0.5489, -0.9256, -1.1926]]),
    x: Batch(
           c: tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218]]),
       ),
)

In [13]:
%timeit b.a = new_value

653 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.4 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

320 µs ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

299 µs ± 5.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9553ed85e0>
├── 'a' --> tensor([[[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]],
│           
│                   [[-0.0925, -0.6357,  1.1402],
│                    [-2.0423, -1.1758,  0.9877]]])
└── 'x' --> <FastTreeValue 0x7f9552e0cbb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.2 µs ± 976 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9552e0cd90>
├── 'a' --> tensor([[-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877],
│                   [-0.0925, -0.6357,  1.1402],
│                   [-2.0423, -1.1758,  0.9877]])
└── 'x' --> <FastTreeValue 0x7f9552e0c9a0>
    └── 'c' --> tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                        [-0.8656,  1.7485, -2.4316, -1.8778],
                 

In [23]:
%timeit t_cat(trees)

46.4 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

102 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2343,  0.2786,  1.5216, -1.9536],
                       [-0.8656,  1.7485, -2.4316, -1.8778],
                       [-1.3291,  0.1364,  1.3137, -1.9218]],
              
                      [[ 0.2343,  0.2786,  1.5216, -1.9536],
                       [-0.8656,  1.7485, -2.4316, -1.8778],
                       [-1.3291,  0.1364,  1.3137, -1.9218]],
              
                      [[ 0.2343,  0.2786,  1.5216, -1.9536],
                       [-0.8656,  1.7485, -2.4316, -1.8778],
                       [-1.3291,  0.1364,  1.3137, -1.9218]],
              
                      [[ 0.2343,  0.2786,  1.5216, -1.9536],
                       [-0.8656,  1.7485, -2.4316, -1.8778],
                       [-1.3291,  0.1364,  1.3137, -1.9218]],
              
                      [[ 0.2343,  0.2786,  1.5216, -1.9536],
                       [-0.8656,  1.7485, -2.4316, -1.8778],
                       [-1.3291,  0.1364,  1.3137, -1.9218]],

In [26]:
%timeit Batch.stack(batches)

116 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218],
                      [ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218],
                      [ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218],
                      [ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218],
                      [ 0.2343,  0.2786,  1.5216, -1.9536],
                      [-0.8656,  1.7485, -2.4316, -1.8778],
                      [-1.3291,  0.1364,  1.3137, -1.9218],
                      [ 0.2343,  0.2786,  1.5216, -1.9536],
                   

In [28]:
%timeit Batch.cat(batches)

215 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

774 µs ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
